### Workflow

1. Per la prima stringa si suggeriscono solo le skill che iniziano per la stringa inserita fino a quel momento.\
TODO: decidere quali mostrare, quelle più lunghe? a caso?


2. Per le successive stringe si considerano le skill inserite fino a quel momento: partendo dalla stringa inserita si ricavano le skill che contengono quella stringa, come in 1, poi per ognuna di queste si calcola la similarità del coseno con le skill già inserite e si suggerisce la skill con la media delle similarità più altra\
TODO: decidere quante skill mostrare. Usare la gerarchia delle skill come altra metrica di suggerimento, ad esempio in base alla distanza dal primo antenato in comune


3. suggerire anche una skill che non contenga la stringa inserita fino a quel momento ma che sia semplicemente la più simile con quelle inserite precedentemente, calcolata come per 2


4. Gestire gli errori di typo: individuarli e suggerire un'alternativa

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from bisect import bisect_left
import ipywidgets as widgets
import difflib
from gensim.models import FastText, WordEmbeddingSimilarityIndex
from IPython.core.display import display, HTML
from gensim.similarities import SparseTermSimilarityMatrix, MatrixSimilarity, SparseMatrixSimilarity

In [ ]:
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")

cos_sim_matrix = pd.DataFrame(cosine_similarity(np.array(list(b.values()))),
                                   columns=list(b.keys()),
                                   index=list(b.keys()))

In [257]:
t = SuggestsTextArea(vectors, cos_sim_matrix)
t.show_form()

In [256]:
class SuggestsTextArea:
    
    def __init__(self, model, cos_sim_matrix):
        # peso dell'input inserito nel calcolo della similarità finale
        self.alpha = 0.8
        
        self.skills_list = pd.read_excel("data/2020_06_09 Allocation to ONET.xlsx")['escoskill_level_3']
        self.vectors = model #FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")
        self.cos_sim_matrix = cos_sim_matrix #pd.DataFrame(cosine_similarity(self.batch), 
        #                                   columns=list(self.vectors.keys())[0:100],
        #                                   index=list(self.vectors.keys())[0:100])
        
        #layout button
        self.layout_button=Layout(width='25%', height='50px')
        
        # lista dei bottoni
        self.suggest_buttons = list()
        self.suggest_buttons_context = list()
        
        # set up della text area
        self.textArea = widgets.Textarea(
            value='',
            placeholder='Type something',
            disabled=False,
            tooltip='Enter the name of the Text field',
            height='90px',
            layout=widgets.Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
        )
        
        # set up della box per i bottoni dei suggerimenti per l'input
        self.suggests = widgets.HBox(self.suggest_buttons, 
                                     layout=widgets.Layout(flex='0 1 auto', height='100px', width='90%'))
        
        # set up della box per i bottoni dei suggerimenti solo per il context
        self.suggests_context = widgets.HBox(self.suggest_buttons_context, 
                                             layout=widgets.Layout(height='100px', width='90%'))
        
        
        # set up dell'intero form
        self.form = widgets.VBox([widgets.VBox([widgets.Label(value='Skills:'), self.textArea]), 
                                 widgets.HBox([widgets.Label(value='Suggests:'), self.suggests]),
                                 widgets.HBox([widgets.Label(value='Similar skills:'), 
                                               self.suggests_context])])
    
    
    def find_sub_sting(self, word, words_list, context, limit=4):
        '''
        Find all skill that contain the new string insert by user
        '''
        return sorted([(w , self.vectors.wv.similarity(word, w)) for w in words_list if w not in context], 
                      key=lambda tup: tup[1], 
                      reverse=True)[:limit]
        #return [w for w in sorted(words_list) if w.lower().startswith(word.lower())]
    
    
    def get_best_similarity_skill(self, context, new_input):
        '''
        Get the four most similarity skills respect of the user's input
        '''
        # similarità tra input e lista delle skills
        find_suggests = self.find_sub_sting(new_input, self.skills_list, context, limit=4)
        res = dict()
        res_sim = dict()
        
        # calcolo delle medie delle similarità tra le skills candidate come suggerimenti e le skills già inserite
        for skill in [s[0] for s in find_suggests]:
            res[skill] = np.array([self.vectors.wv.similarity(skill, c) for c in context]).mean()
        
        # calcolo della similarità aggregata
        for i in range(0, 4):
            skill = find_suggests[i][0]
            final_sim = (self.alpha * find_suggests[i][1]) + ((1-self.alpha) * res[skill])
            res_sim[skill] = final_sim
        return {k: v for k, v in sorted(res_sim.items(), key=lambda item: item[1], reverse=True)}
        #(a * sim(parola_corrente, parole_excel) + (1-a) * media(similarità(parola_precedente, parole, excel)))
        #candidate_skill = self.cos_sim_matrix.loc[context, find_suggests]
        #return candidate_skill.mean(axis = 0).sort_values(ascending=False)
        #return candidate_skill.mean(axis = 0).idxmax(), candidate_skill.mean(axis = 0).max()
        
    
    def show_form(self):
        '''
        Show the text area widget
        '''
        display(self.form)
        self.textArea.observe(self.suggests_manager, names='value')
    
    def suggests_manager(self, widget):
        '''
        Main function of the class, manage the suggestions in different case
        '''
        self.close_suggest_buttons()
        old_input = widget['old']
        new_input = widget['new'].split(' ')[-1]
        context = old_input.split(', ')[:-1]
        if len(new_input)>3:
            if context == []:
                find_suggests = self.find_sub_sting(new_input, self.skills_list, context, limit=4)                    

                for sug in find_suggests:

                    b = widgets.Button(
                        description=f'{sug[0]} - {round(sug[1]*100, 2)}%',
                        disabled=False,
                        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip=f'{sug[0]} - {round(sug[1]*100, 2)}%',
                        layout=self.layout_button
                    )

                    b.on_click(self.add_skill)
                    self.suggest_buttons.append(b)
            else:
                sug = self.get_best_similarity_skill(context, new_input)
                for index, value in sug.items():
                    b = widgets.Button(
                        description=f'{index} - \n{round(value*100, 2)}%', # fix negative similarity
                        disabled=False,
                        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip=f'{index} - {round(value*100, 2)}%',
                        layout=self.layout_button
                    )

                    b.on_click(self.add_skill)
                    self.suggest_buttons.append(b)

                #display(widgets.HBox(self.suggest_buttons))
        self.suggests.children=tuple(self.suggest_buttons)
                

    def add_skill(self, btn_object):
        '''
        Button click handler, add the skill pressed to the text area
        '''
        skill_list = self.textArea.value.split(', ')
        skill_list[-1] = btn_object.description.split(" -")[0]
        new_contest = ', '.join(skill_list) + ', '
        self.textArea.value = new_contest
        self.close_suggest_buttons()
        self.close_suggest_buttons_context()
        self.suggest_interest_skill(skill_list)
        
        
    def suggest_interest_skill(self, context):
        ll = list()
        for word in [w for w in cos_sim_matrix.columns if w not in context]:
            ll.append((word, cos_sim_matrix.loc[word, context].mean()))
        find_suggests = sorted(ll, key=lambda tup: -tup[1])[:4]
        
        for sug in find_suggests:
            b = widgets.Button(
                description=f'{sug[0]} - {round(sug[1]*100, 2)}%',
                disabled=False,
                button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                tooltip=f'{sug[0]} - {round(sug[1]*100, 2)}%',
                layout=self.layout_button
                )
            b.on_click(self.add_skill)
            self.suggest_buttons_context.append(b)
        self.suggests_context.children=tuple(self.suggest_buttons_context)
        return sug

    
    def close_suggest_buttons(self):
        '''
        Remove button of old suggestions
        '''
        for w in self.suggest_buttons:
            w.close()
        self.suggest_buttons = list()
    
    
    def close_suggest_buttons_context(self):
        '''
        Remove button of old suggestions
        '''
        for w in self.suggest_buttons_context:
            w.close()
        self.suggest_buttons_context = list()
    
    

In [3]:
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")

/home/simone/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [142]:
matrix = SparseMatrixSimilarity(b.values(), num_features=len(b.keys()))

In [57]:
SparseMatrixSimilarity.save(matrix, "ss")

In [125]:
print(matrix.index)

  (0, 0)	2.0627089
  (0, 1)	1.0952042
  (0, 2)	0.33151388
  (0, 3)	2.4008334
  (0, 4)	-1.2678887
  (0, 5)	-0.9272374
  (0, 6)	-2.630406
  (0, 7)	1.7138474
  (0, 8)	1.0676862
  (0, 9)	2.7220395
  (0, 10)	1.0334668
  (0, 11)	0.18803456
  (0, 12)	-0.39986444
  (0, 13)	-0.5320387
  (0, 14)	-0.56712043
  (0, 15)	-2.4658859
  (0, 16)	-1.0781564
  (0, 17)	-0.73728895
  (0, 18)	2.782699
  (0, 19)	-2.0474477
  (0, 20)	0.6597559
  (0, 21)	-1.4508687
  (0, 22)	-1.4879175
  (0, 23)	-1.7104709
  (0, 24)	2.0996559
  :	:
  (2318, 25)	-0.03162113
  (2318, 26)	-1.7446702
  (2318, 27)	1.59824
  (2318, 28)	2.4698663
  (2318, 29)	-1.8482527
  (2318, 30)	0.35833976
  (2318, 31)	-1.7721348
  (2318, 32)	-0.5337102
  (2318, 33)	-1.0553359
  (2318, 34)	0.52250683
  (2318, 35)	-4.828547
  (2318, 36)	0.5340927
  (2318, 37)	0.22976083
  (2318, 38)	2.0193362
  (2318, 39)	-0.7197275
  (2318, 40)	2.6645496
  (2318, 41)	2.0446312
  (2318, 42)	1.293121
  (2318, 43)	3.2427478
  (2318, 44)	-0.5589939
  (2318, 45)	2.1738

In [116]:
matrix.get_similarities([(1,2)])

array([ 2.1904085 ,  1.7696383 , -0.38618374, ...,  4.6632266 ,
       -1.4279553 ,  4.2544775 ], dtype=float32)

In [4]:
aa = pd.read_excel("data/2020_06_09 Allocation to ONET.xlsx")['escoskill_level_3'].tolist()
aa

['adapt to change',
 'work as a team',
 'use a computer',
 'teamwork principles',
 'English',
 'use microsoft office',
 'assist customers',
 'create solutions to problems',
 'communication',
 'adapt to changing situations',
 'use office systems',
 'show responsibility',
 'think proactively',
 'office software',
 'customer service',
 'project management',
 'develop strategy to solve problems',
 'team building',
 'work independently',
 'tolerate stress',
 'think creatively',
 'adjust priorities',
 'manage time',
 'work in teams',
 'prioritise tasks',
 'analyse problems for opportunities',
 'lead a team',
 'quality standards',
 'office administration',
 'demonstrate enthusiasm',
 'economics',
 'provide leadership',
 'computer programming',
 'provide information',
 'use spreadsheets',
 'online analytical processing',
 'develop creative ideas',
 'brainstorm ideas',
 'use communication techniques',
 'plan teamwork',
 'business ICT systems',
 'sales activities',
 'follow company standards',
 

In [140]:
type(b.values())

dict_values

In [4]:
vectors.wv.vocab['adapt to change']

KeyError: 'adapt to change'

In [5]:
b = {k: vectors.wv[k] for k in aa}

In [62]:
len(b.keys())

array(['adapt to change', 'work as a team', 'use a computer', ...,
       'on board hazards',
       'control objectives for information and related technology',
       'attend design meetings'], dtype='<U66')

In [57]:
from gensim.matutils import Dense2Corpus
index = MatrixSimilarity(Dense2Corpus(vectors.wv.syn0.T))

/home/simone/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [66]:
from sklearn.metrics.pairwise import cosine_similarity
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")
cosine_similarity(np.array(list(b.values())))

array([[ 1.0000001 ,  0.30536905, -0.02451085, ...,  0.12244662,
         0.46511036,  0.07769969],
       [ 0.30536905,  1.        , -0.07481783, ...,  0.05608097,
         0.23602311,  0.30638945],
       [-0.02451085, -0.07481783,  1.        , ...,  0.11000007,
         0.3065195 ,  0.10185154],
       ...,
       [ 0.12244662,  0.05608097,  0.11000007, ...,  1.0000001 ,
         0.1186327 ,  0.35572955],
       [ 0.46511036,  0.23602311,  0.3065195 , ...,  0.1186327 ,
         1.0000001 ,  0.25273737],
       [ 0.07769969,  0.30638945,  0.10185154, ...,  0.35572955,
         0.25273737,  0.99999994]], dtype=float32)

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")
cosine_similarity(np.array(list(b.values())))
cos_sim_matrix = pd.DataFrame(cosine_similarity(np.array(list(b.values()))),
                                   columns=list(b.keys()),
                                   index=list(b.keys()))

In [119]:
%%time
#from operator import itemgetter
context = cos_sim_matrix.columns[0:2]
ll = list()
for word in [w for w in cos_sim_matrix.columns if w not in context]:
    ll.append((word, cos_sim_matrix.loc[word, context].mean()))
sorted(ll, key=lambda tup: tup[1])[:4]

CPU times: user 1.83 s, sys: 16 ms, total: 1.84 s
Wall time: 1.8 s


[('hearing loss', -0.38584816455841064),
 ('Estonian', -0.3273324966430664),
 ('hearing aids', -0.3229963481426239),
 ('types of wood', -0.28223955631256104)]

In [74]:
cos_sim_matrix

,adapt to change,work as a team,use a computer,teamwork principles,English,use microsoft office,assist customers,create solutions to problems,communication,adapt to changing situations,...,follow airport safety procedures,apply change management,adjust clothes,ensure efficient baggage handling,product data management,sell hardware,provide assistive technology,on board hazards,control objectives for information and related technology,attend design meetings
adapt to change,1.000000,0.305369,-0.024511,0.425366,0.062302,0.075967,0.232879,0.403997,0.431470,0.784645,...,0.088773,0.487714,0.202321,0.270042,0.389113,0.121583,0.126435,0.122447,0.465110,0.077700
work as a team,0.305369,1.000000,-0.074818,0.147355,0.095329,0.128273,0.342489,0.356652,0.235979,0.190077,...,0.017533,0.135197,-0.026944,0.243395,0.306660,0.082453,0.209561,0.056081,0.236023,0.306389
use a computer,-0.024511,-0.074818,1.000000,0.301733,0.401523,0.388473,0.016860,0.185535,0.466150,0.095024,...,0.122054,0.283296,-0.224272,0.217526,0.345534,0.456069,0.273269,0.110000,0.306520,0.101852
teamwork principles,0.425366,0.147355,0.301733,1.000000,0.208126,0.008501,-0.028138,0.273564,0.564180,0.359767,...,0.216186,0.351148,-0.111817,0.161343,0.364832,-0.036258,-0.009770,0.010772,0.395168,0.127702
English,0.062302,0.095329,0.401523,0.208126,1.000000,0.178999,-0.061851,-0.140542,0.594137,0.260248,...,0.074013,0.163728,0.157660,0.062051,-0.015155,0.083693,-0.072765,-0.054197,-0.075331,0.165147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sell hardware,0.121583,0.082453,0.456069,-0.036258,0.083693,0.401038,0.325111,0.228782,0.231796,0.161606,...,0.131213,0.260966,0.081272,0.361417,0.368458,1.000000,0.313351,0.119203,0.164580,0.109877
provide assistive technology,0.126435,0.209561,0.273269,-0.009770,-0.072765,0.236536,0.582770,0.390335,0.188317,0.160738,...,0.157900,0.263235,-0.228761,0.373144,0.271379,0.313351,1.000000,-0.006843,0.368096,0.307158
on board hazards,0.122447,0.056081,0.110000,0.010772,-0.054197,-0.036496,0.261189,0.169045,-0.046536,0.213786,...,0.539554,0.109508,0.288844,0.356929,0.064573,0.119203,-0.006843,1.000000,0.118633,0.355730
control objectives for information and related technology,0.465110,0.236023,0.306520,0.395168,-0.075331,0.010821,0.365176,0.398812,0.469975,0.375752,...,0.265700,0.639479,-0.147877,0.463509,0.656394,0.164580,0.368096,0.118633,1.000000,0.252737


In [45]:
index[[(1,2), (1,2)]]

array([ 1.5488527 ,  1.2513232 , -0.27307314, ...,  3.297399  ,
       -1.0097169 ,  3.00837   ], dtype=float32)

In [157]:
len(vectors.wv[aa])

2319

In [157]:
similarity_matrix = cosine_similarity(vectors.wv.vectors)

MemoryError: 

In [135]:
termsim_index = WordEmbeddingSimilarityIndex(vectors.wv)
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, vectors.wv.vocab)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [71]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

btn.on_click(on_bttn_clicked)
display(vb, btn)

list(vb.children)

Button(description='Add', style=ButtonStyle())

[Text(value='1'), Text(value='2')]

In [89]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, HBox, VBox

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

b1 = widgets.Button(
    description=f'eeeee',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
b2 = widgets.Button(
    description=f'ewwwww',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

box = VBox([b1, b2])

btn = wg.Button(description = 'Add') 
def on_bttn_clicked(btn):        
    box.layout.visibility = 'visible'

btn.on_click(on_bttn_clicked)

dele = wg.Button(description = 'del') 
def dee(btn):
    print("ss")
    box.layout.visibility = 'hidden'

dele.on_click(dee)

form_items = [
    Box([Label(value='Information'),
         Textarea()], layout=form_item_layout),
    Box([Label(value='Suggests'),
         box], layout=form_item_layout),
    Box([Label(value=''),
         btn], layout=form_item_layout),
    Box([Label(value=''),
         dele], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))


display(form)

#resetButton.layout.visibility = 'hidden'

Box(children=(Box(children=(Label(value='Information'), Textarea(value='')), layout=Layout(display='flex', fle…

In [104]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

btn.on_click(on_bttn_clicked)
display(vb, btn)

list(vb.children)

Button(description='Add', style=ButtonStyle())

[Text(value='1'), Text(value='2')]